In [ ]:
#| default_exp model.detector

In [ ]:
#| hide
from nbdev.showdoc import *
%load_ext autoreload
%autoreload 2

In [ ]:
#|export
import cv2
import timm
import torch
import pickle
import random
import torch.nn as nn
from license_plate_detection.utils import *
from fastai.vision.all import *
from torchvision.ops import roi_align
from license_plate_detection.fpn import *
from license_plate_detection.model import *

In [ ]:
#| export
class FasterRCNN(nn.Module):
    def __init__(self, backbone, rpn, roi):
        super().__init__()
        
        #TODO add fpn
        self.backbone= backbone
        self.rpn_head= rpn
        self.roi_head= roi
        # self.fpn= fpn
    
    @classmethod
    def init_model(cls, arch, dls, ratios, scales, device='cuda'):
        xb= dls.one_batch()[0].shape
        dummy_inp= torch.empty(xb)
        backbone= timm.create_model(arch, features_only=True, out_indices=[2])
        with torch.no_grad(): dummy_ft= backbone(dummy_inp)[0].shape
        stride= torch.Tensor([xb[-2:]])/torch.Tensor([dummy_ft[-2:]])
        num_classes= len(dls.vocab)-1
        rpn=RPN(dummy_ft[1], stride, ratios, scales, dummy_ft[-2:], device)
        roi= ROI(dummy_ft[1], num_classes)
        return cls(backbone,rpn,roi)
        
    def forward(self, x):
        x = self.backbone(x)[0]
        rpn_reg_feats, rpn_cls_feats, rois= self.rpn_head(x)
        roi_reg_feats, roi_cls_feats= self.roi_head(x, rois)
        return rpn_reg_feats, rpn_cls_feats, rois, roi_reg_feats, roi_cls_feats

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()